In [12]:
import pandas as pd
import numpy as np
import warnings
import pickle
import matplotlib.pyplot as plt
warnings.filterwarnings("ignore")

In [13]:
import os
import sys
module_path = os.path.abspath(os.path.join('../../utils'))
if module_path not in sys.path:
    sys.path.append(module_path)

from helper_functions import *

In [14]:
nb_variables = {'shift': -1}

In [15]:
test = pd.read_csv('Data/modified_test_data.csv', index_col = 0)

In [16]:
test = n_step_shift(test, nb_variables['shift'])

In [17]:
xgb_reg = pickle.load(open('model.pkl', 'rb'))

In [18]:
preds = xgb_reg.predict(test)
preds[preds < 0] = 0
preds = np.round(preds).astype('int32')

In [19]:
preds = fill_future_predictions(preds, nb_variables['shift'])
preds.shape

(156,)

In [20]:
submission = pd.read_csv('../../../dengue_features_test.csv', index_col = 0)
submission = submission.loc['iq']
submission = submission.loc[:, ['year', 'weekofyear']]
submission.head()

,year,weekofyear
city,,
iq,2010,26
iq,2010,27
iq,2010,28
iq,2010,29
iq,2010,30


In [21]:
preds = pd.Series(preds.reshape(-1))
preds.index = submission.index
submission = pd.concat([submission, preds], axis = 1)
submission.columns = ['year', 'weekofyear', 'total_cases']
submission.head()

,year,weekofyear,total_cases
city,,,
iq,2010,26,6
iq,2010,27,2
iq,2010,28,5
iq,2010,29,4
iq,2010,30,2


In [22]:
submission.to_csv('sj_XGBoost_predictions.csv')